In [1]:
!pip install langchain-community langchain-core pypdf langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-spli

In [2]:
from langchain_community.document_loaders import PyPDFLoader

# Example file
from google.colab import files
uploaded = files.upload()  # upload a PDF from computer

file_path = list(uploaded.keys())[0]  # get uploaded filename

# Load the PDF
loader = PyPDFLoader(file_path)
docs = loader.load()

print(f"✅ Loaded {len(docs)} pages.")
print("🔹Sample text:\n", docs[0].page_content[:500])


Saving Agent Based Code Generation.pdf to Agent Based Code Generation.pdf
✅ Loaded 20 pages.
🔹Sample text:
 SMARTER MULTI-AGENT CODE GENERATION
INSPIRED BY SDLC PRINCIPLES
210042107
210042170
210042172
Md Hasibur Rahman Alif
Hasibul Islam Nirjhar
Adid Al Mahamud Shazid
Design Project II - SWE 4606 
Network and Data Analysis Group (NDAG)Dr. Md. Azam Hossain 
Associate Professor
Supervised By


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     # each chunk = 1000 characters
    chunk_overlap=200,   # overlap helps preserve context
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

print(f"✅ Split into {len(all_splits)} smaller pieces.")
print(all_splits[0].page_content[:200])


✅ Split into 20 smaller pieces.
SMARTER MULTI-AGENT CODE GENERATION
INSPIRED BY SDLC PRINCIPLES
210042107
210042170
210042172
Md Hasibur Rahman Alif
Hasibul Islam Nirjhar
Adid Al Mahamud Shazid
Design Project II - SWE 4606 
Network 


In [4]:
pip install -qU langchain-huggingface

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[-0.02148606814444065, 0.02424011006951332, -0.018047209829092026, -0.03897859901189804, -0.03339795768260956, -0.04823237657546997, 0.0600193589925766, -0.002922870684415102, 0.01209822203963995, 0.01615728810429573]


In [7]:
pip install -qU langchain-core

In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

# Store all text chunks
vector_store.add_documents(all_splits)

print("✅ Vector store ready!")


✅ Vector store ready!


In [11]:
query = "How many pages are there in the slide?"
results = vector_store.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"🔹Result {i}:")
    print(doc.page_content[:400], "\n---\n")

print



🔹Result 1:
333
OVERVIEW 
---

🔹Result 2:
151515
RESEARCHCHALLENGES 
---



In [14]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many types of code generation?",
        "What is multiagent?",
    ],
)

[[Document(id='8889c200-8167-4652-8d02-dbaa3d67a2ae', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-10-23T15:52:20+00:00', 'title': 'Agent Based Code Generation', 'moddate': '2025-10-23T15:52:16+00:00', 'keywords': 'DAG2MxMofew,BAEO_xsYlic,0', 'author': 'Adid- Al -Mahamud Shazid 210042172', 'source': 'Agent Based Code Generation.pdf', 'total_pages': 20, 'page': 3, 'page_label': '4', 'start_index': 0}, page_content='What is Code Generation?\nTranslating natural-language requirements into\nexecutable code.\nTraditionally, it focuses on automating repetitive or\ntemplated programming tasks.\nIn modern research, AI-driven code generation\nseeks to understand intent and produce\nfunctionally correct, optimized code.\n4\nNatural-language specification\xa0\xa0 →  \xa0Executable Program\nStatement\xa0 |\xa0 Function\xa0 |\xa0 Repository')],
 [Document(id='de8ee3a9-3562-4c2f-b1f5-ed14e13d11b6', metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-1